Model Training File

importing required libraries

In [5]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
import json
import time
import pandas as pd
import seaborn as sns

c:\Users\sara_\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


setting up api key as environment variable

In [6]:
load_dotenv('api_key.env')

True

In [7]:
api_key = os.environ.get('GENAI_API_KEY')

In [8]:
genai.configure(api_key=api_key)

list of tuned models

In [46]:
for model_info in genai.list_tuned_models():
    print(model_info.name)

tunedModels/mercury-adhhc1b6yksy
tunedModels/mercury-vpmzvtplbrrt
tunedModels/mercury-t7ytex3p625n
tunedModels/mercury-4carel57u8nz
tunedModels/mercury-x71oyrk7fgc6


reading data from json file

In [47]:
def load_jsonl_data(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

In [48]:
training_data = load_jsonl_data("query_response.jsonl")

In [49]:
print(training_data)

[{'query': "What's your name?", 'response': "Hello! I'm Mercury, your go-to assistant for any company-related queries. Think of me as your personal guide to navigating company processes."}, {'query': 'What exactly do you do?', 'response': "I'm here to make your work life easier. I efficiently handle employee queries, provide quick access to company policies and resources, and curate answers based on collective knowledge from the company forum."}, {'query': 'How can you help me?', 'response': 'I can assist with understanding policies, navigating processes, finding resources, and even answering day-to-day queries. If youâ€™re stuck, just ask me!'}, {'query': 'Are you available 24/7?', 'response': 'Absolutely! Iâ€™m always here to assist you, whether itâ€™s day or night, weekday or weekend. Just let me know what you need.'}, {'query': 'Can you handle technical issues?', 'response': 'I can guide you through common troubleshooting steps or connect you with the right team if itâ€™s something

specifying base model for tuning

In [55]:
base_model = 'models/gemini-1.5-flash-001-tuning'

tuning the model

In [56]:
operation = genai.create_tuned_model(
    display_name = "Mercury",
    source_model = base_model,
    epoch_count = 10,
    batch_size =2,
    learning_rate = 0.0001,
    training_data = training_data,
    input_key = 'query',
    output_key = 'response'
)
for status in operation.wait_bar():
    time.sleep(10)

100%|██████████| 450/450 [42:18<00:00,  5.64s/it]


In [1]:
result = operation.result()
print(result)

NameError: name 'operation' is not defined

storing tuned model in 'model'

In [9]:
model = genai.GenerativeModel(model_name='tunedModels/mercury-w0aigfuv20g3')

tracking loss in model training

In [10]:
snapshots = pd.DataFrame(result.tuning_task.snapshots)
sns.lineplot(data=snapshots, x='epoch', y='mean_loss')

NameError: name 'result' is not defined

example:

In [13]:

result = model.generate_content("what do you do?")
print(result.text)

I'm your go-to AI for any company-related information. Need data, a report, or even a quick answer? Just ask me, and I'll provide it in a flash. Think of me as your personal assistant, always ready to help you work smarter, not harder.


defining it as a function

In [90]:
def generate_response(prompt):
    model = genai.GenerativeModel(model_name = result.name)
    response = model.generate_content(prompt)
    return response.text

In [91]:
generate_response('hello')

'Hello! What can I do for you today? Feel free to ask me anything.â'